In [23]:
import json
import pandas as pd

In [24]:
heart = pd.read_json('~/Desktop/py129/self-viz/sample_heart.json')

# Empty lists for heartbeats/min and Fitbit 'Confidence' value
bpm_list = []
conf_list = []

# Populates empty lists with respective Fitbit values
for i in heart['value']:
    bpm_list.append(i['bpm'])
    conf_list.append(i['confidence'])

# Creates new pandas dataframes from populated lists
bpm_df = pd.DataFrame(bpm_list)
conf_df = pd.DataFrame(conf_list)

# Assigns dataframes as new series/columns in original dataframe
heart['BPM'] = bpm_df
heart['Confidence'] = conf_df

# Exports dataframe as CSV
heart[['dateTime','BPM','Confidence']].to_csv(('~/Desktop/py129/self-viz/sample_heart.csv'))


In [25]:
# Reads JSON file and renames column for Calories burned per minute
calor = pd.read_json('~/Desktop/py129/self-viz/sample_calories.json')
calor.rename(columns={'value':'Calories'}, inplace=True)

# Exports dataframe as CSV
calor.to_csv(('~/Desktop/py129/self-viz/sample_calories.csv'))


In [26]:
heart = pd.read_csv('~/Desktop/py129/self-viz/denver_trip.csv')

In [28]:
import psycopg2

try:
    # Creates connection to new DB, will create DB if nonexistant
    #conn = sqlite3.connect('heartcaltrip')
    conn = psycopg2.connect(
        database='heartcaltrip',
        user='steve_wortmann',
        password='paigek#624',
        host='localhost'
    )
    
    # Cursor will give us result set
    cursor = conn.cursor()
    print("Connected")
    
    hearttable_sql='''
                    CREATE TABLE IF NOT EXISTS
                        heart(
                            time_id INTEGER PRIMARY KEY,
                            dateTime TIMESTAMP,
                            Bpm INTEGER,
                            Confidence INTEGER
                        );
                '''
    cursor.execute(hearttable_sql)
    conn.commit()
    print("Created HEART Table")
    
    calorietable_sql='''
                    CREATE TABLE IF NOT EXISTS
                        calorie(
                            time_id INTEGER PRIMARY KEY,
                            dateTime TIMESTAMP,
                            Calories FLOAT
                        );
                '''
    cursor.execute(calorietable_sql)
    conn.commit()
    print("Created CALORIE Table")

    triptable_sql='''
                    CREATE TABLE IF NOT EXISTS
                        trip(
                            time_id INTEGER PRIMARY KEY,
                            name TEXT,
                            address TEXT,
                            description TEXT,
                            TimeSpan_begin TIMESTAMP,
                            TimeSpan_end TIMESTAMP,
                            longitude FLOAT,
                            latitude FLOAT,
                            LineString_coordinates TEXT
                        );
                '''
    cursor.execute(triptable_sql)
    conn.commit()
    print("Created TRIP Table")
    
except psycopg2.Error as error:
        print("Error")
        print(error)
# Always close up our resources
finally:
    if cursor:
        cursor.close()
        if conn:
            conn.close()

Connected
Created HEART Table
Created CALORIE Table
Created TRIP Table


In [29]:
def close_DB_resources(dbconn, cursor):
    try:
        if dbconn:
            dbconn.close()
        if cursor:
            cursor.close()
        print("closed resources")
    except psycopg2.Error as err:
        ("Error closing resources")

In [30]:
# Utility for handling DB errors
def handle_DB_error(dbconn, cursor):
    if dbconn:
        try:
            dbconn.rollback()
            print("Rolled back transation")
        except psycopg2.Error as error:
            print("Error rolling back transaction")
        finally:
            close_DB_resources(dbconn, cursor)
            dbconn = None
            return dbconn

In [31]:
# Utility DB connection and cursor function to return our access objects
def connect_SQLite():
    try:
        dbconn = psycopg2.connect(
            database='heartcaltrip',
            user='steve_wortmann',
            password='paigek#624',
            host='localhost'
        )
        cursor = dbconn.cursor()
        print("Connected to heartcaltrip")
    except psycopg2.Error as error:
        print("Error opening database")
        dbconn = handle_DB_error(dbconn, cursor)
    return dbconn, cursor

In [32]:
import csv

heart_insert = '''INSERT INTO heart (time_id, dateTime, Bpm, Confidence)
                  VALUES (%s,%s,%s,%s);
                '''
# Retrieve my connection and cursor for use in all my inserts
dbconn, cursor = connect_SQLite()
with open('sample_heart.csv') as file:
    time_stamps = csv.DictReader(file)
    value_id = 0
    for time in time_stamps:
        value_id += 1
        record = (value_id, time['dateTime'], time['BPM'], time['Confidence'])
        print(record)
        cursor.execute(heart_insert, record)
        dbconn.commit()

close_DB_resources(dbconn, cursor)

Connected to heartcaltrip
(1, '2020-10-04 06:00:01', '130', '1')
(2, '2020-10-04 06:00:06', '132', '1')
(3, '2020-10-04 06:00:21', '133', '1')
(4, '2020-10-04 06:00:31', '132', '2')
(5, '2020-10-04 06:00:41', '131', '2')
(6, '2020-10-04 06:00:46', '129', '1')
(7, '2020-10-04 06:00:51', '130', '1')
(8, '2020-10-04 06:00:56', '132', '2')
(9, '2020-10-04 06:01:01', '130', '2')
(10, '2020-10-04 06:01:11', '128', '2')
(11, '2020-10-04 06:01:16', '126', '2')
(12, '2020-10-04 06:01:31', '127', '2')
(13, '2020-10-04 06:01:36', '130', '2')
(14, '2020-10-04 06:01:41', '131', '2')
(15, '2020-10-04 06:01:46', '130', '2')
(16, '2020-10-04 06:01:51', '129', '2')
(17, '2020-10-04 06:01:56', '130', '2')
(18, '2020-10-04 06:02:01', '129', '2')
(19, '2020-10-04 06:02:06', '127', '1')
(20, '2020-10-04 06:02:11', '126', '1')
(21, '2020-10-04 06:02:16', '125', '1')
(22, '2020-10-04 06:02:21', '125', '2')
(23, '2020-10-04 06:02:26', '126', '2')
(24, '2020-10-04 06:02:31', '127', '2')
(25, '2020-10-04 06:02:

(207, '2020-10-04 06:24:05', '102', '3')
(208, '2020-10-04 06:24:10', '94', '3')
(209, '2020-10-04 06:24:15', '97', '3')
(210, '2020-10-04 06:24:30', '97', '3')
(211, '2020-10-04 06:24:35', '98', '3')
(212, '2020-10-04 06:24:40', '99', '2')
(213, '2020-10-04 06:24:50', '100', '2')
(214, '2020-10-04 06:24:55', '101', '2')
(215, '2020-10-04 06:25:00', '110', '3')
(216, '2020-10-04 06:25:05', '105', '3')
(217, '2020-10-04 06:25:10', '101', '3')
(218, '2020-10-04 06:25:15', '100', '2')
(219, '2020-10-04 06:25:25', '101', '2')
(220, '2020-10-04 06:25:30', '102', '2')
(221, '2020-10-04 06:25:35', '108', '2')
(222, '2020-10-04 06:25:40', '107', '2')
(223, '2020-10-04 06:25:50', '103', '3')
(224, '2020-10-04 06:25:55', '101', '3')
(225, '2020-10-04 06:26:00', '101', '2')
(226, '2020-10-04 06:26:05', '102', '2')
(227, '2020-10-04 06:26:15', '104', '2')
(228, '2020-10-04 06:26:20', '106', '2')
(229, '2020-10-04 06:26:25', '108', '2')
(230, '2020-10-04 06:26:30', '106', '2')
(231, '2020-10-04 06:

(414, '2020-10-04 06:51:59', '83', '3')
(415, '2020-10-04 06:52:09', '82', '3')
(416, '2020-10-04 06:52:24', '83', '3')
(417, '2020-10-04 06:52:29', '82', '3')
(418, '2020-10-04 06:52:34', '83', '3')
(419, '2020-10-04 06:52:44', '84', '3')
(420, '2020-10-04 06:52:49', '84', '3')
(421, '2020-10-04 06:53:04', '83', '3')
(422, '2020-10-04 06:53:19', '83', '3')
(423, '2020-10-04 06:53:34', '84', '3')
(424, '2020-10-04 06:53:49', '83', '3')
(425, '2020-10-04 06:53:54', '82', '3')
(426, '2020-10-04 06:54:04', '83', '3')
(427, '2020-10-04 06:54:19', '83', '3')
(428, '2020-10-04 06:54:24', '84', '3')
(429, '2020-10-04 06:54:29', '83', '3')
(430, '2020-10-04 06:54:44', '83', '3')
(431, '2020-10-04 06:54:49', '84', '3')
(432, '2020-10-04 06:55:04', '84', '3')
(433, '2020-10-04 06:55:09', '83', '3')
(434, '2020-10-04 06:55:19', '82', '3')
(435, '2020-10-04 06:55:29', '83', '3')
(436, '2020-10-04 06:55:34', '84', '3')
(437, '2020-10-04 06:55:44', '85', '3')
(438, '2020-10-04 06:55:49', '84', '3')


(638, '2020-10-04 07:24:17', '82', '3')
(639, '2020-10-04 07:24:32', '81', '3')
(640, '2020-10-04 07:24:37', '82', '3')
(641, '2020-10-04 07:24:52', '81', '3')
(642, '2020-10-04 07:25:07', '82', '3')
(643, '2020-10-04 07:25:22', '82', '3')
(644, '2020-10-04 07:25:37', '82', '3')
(645, '2020-10-04 07:25:52', '82', '3')
(646, '2020-10-04 07:25:57', '81', '3')
(647, '2020-10-04 07:26:02', '82', '3')
(648, '2020-10-04 07:26:17', '83', '3')
(649, '2020-10-04 07:26:22', '82', '3')
(650, '2020-10-04 07:26:27', '81', '3')
(651, '2020-10-04 07:26:42', '82', '3')
(652, '2020-10-04 07:26:52', '83', '3')
(653, '2020-10-04 07:27:02', '82', '3')
(654, '2020-10-04 07:27:07', '83', '3')
(655, '2020-10-04 07:27:12', '82', '3')
(656, '2020-10-04 07:27:27', '82', '3')
(657, '2020-10-04 07:27:32', '83', '3')
(658, '2020-10-04 07:27:37', '84', '3')
(659, '2020-10-04 07:27:52', '84', '3')
(660, '2020-10-04 07:28:07', '82', '3')
(661, '2020-10-04 07:28:17', '83', '3')
(662, '2020-10-04 07:28:22', '82', '3')


(990, '2020-10-04 08:10:18', '82', '3')
(991, '2020-10-04 08:10:23', '83', '3')
(992, '2020-10-04 08:10:33', '82', '3')
(993, '2020-10-04 08:10:48', '82', '3')
(994, '2020-10-04 08:10:58', '82', '3')
(995, '2020-10-04 08:11:13', '82', '3')
(996, '2020-10-04 08:11:18', '84', '3')
(997, '2020-10-04 08:11:33', '84', '3')
(998, '2020-10-04 08:11:48', '84', '3')
(999, '2020-10-04 08:11:58', '84', '3')
(1000, '2020-10-04 08:12:03', '86', '3')
(1001, '2020-10-04 08:12:13', '85', '3')
(1002, '2020-10-04 08:12:18', '83', '3')
(1003, '2020-10-04 08:12:33', '83', '3')
(1004, '2020-10-04 08:12:38', '84', '3')
(1005, '2020-10-04 08:12:43', '86', '3')
(1006, '2020-10-04 08:12:48', '88', '3')
(1007, '2020-10-04 08:12:53', '87', '3')
(1008, '2020-10-04 08:12:58', '86', '3')
(1009, '2020-10-04 08:13:03', '85', '3')
(1010, '2020-10-04 08:13:09', '84', '3')
(1011, '2020-10-04 08:13:14', '85', '3')
(1012, '2020-10-04 08:13:29', '86', '3')
(1013, '2020-10-04 08:13:34', '85', '3')
(1014, '2020-10-04 08:13:3

(1394, '2020-10-04 09:05:40', '84', '3')
(1395, '2020-10-04 09:05:45', '83', '3')
(1396, '2020-10-04 09:05:50', '82', '3')
(1397, '2020-10-04 09:05:55', '83', '3')
(1398, '2020-10-04 09:06:00', '82', '3')
(1399, '2020-10-04 09:06:06', '82', '3')
(1400, '2020-10-04 09:06:21', '82', '3')
(1401, '2020-10-04 09:06:36', '82', '3')
(1402, '2020-10-04 09:06:51', '81', '3')
(1403, '2020-10-04 09:06:56', '82', '3')
(1404, '2020-10-04 09:07:01', '81', '3')
(1405, '2020-10-04 09:07:11', '82', '3')
(1406, '2020-10-04 09:07:26', '83', '3')
(1407, '2020-10-04 09:07:31', '82', '3')
(1408, '2020-10-04 09:07:41', '83', '3')
(1409, '2020-10-04 09:07:46', '82', '3')
(1410, '2020-10-04 09:08:01', '82', '3')
(1411, '2020-10-04 09:08:06', '84', '3')
(1412, '2020-10-04 09:08:21', '83', '3')
(1413, '2020-10-04 09:08:31', '84', '3')
(1414, '2020-10-04 09:08:36', '85', '3')
(1415, '2020-10-04 09:08:41', '84', '3')
(1416, '2020-10-04 09:08:46', '83', '3')
(1417, '2020-10-04 09:09:01', '84', '3')
(1418, '2020-10-

(1626, '2020-10-04 09:35:41', '82', '3')
(1627, '2020-10-04 09:35:46', '83', '3')
(1628, '2020-10-04 09:35:56', '85', '3')
(1629, '2020-10-04 09:36:01', '86', '3')
(1630, '2020-10-04 09:36:06', '88', '3')
(1631, '2020-10-04 09:36:11', '91', '3')
(1632, '2020-10-04 09:36:16', '93', '3')
(1633, '2020-10-04 09:36:26', '92', '3')
(1634, '2020-10-04 09:36:31', '91', '3')
(1635, '2020-10-04 09:36:37', '90', '3')
(1636, '2020-10-04 09:36:42', '89', '3')
(1637, '2020-10-04 09:36:47', '88', '3')
(1638, '2020-10-04 09:37:02', '86', '3')
(1639, '2020-10-04 09:37:07', '85', '3')
(1640, '2020-10-04 09:37:22', '85', '3')
(1641, '2020-10-04 09:37:27', '84', '3')
(1642, '2020-10-04 09:37:37', '85', '3')
(1643, '2020-10-04 09:37:52', '85', '3')
(1644, '2020-10-04 09:38:02', '84', '3')
(1645, '2020-10-04 09:38:07', '83', '3')
(1646, '2020-10-04 09:38:12', '82', '3')
(1647, '2020-10-04 09:38:27', '82', '3')
(1648, '2020-10-04 09:38:32', '81', '3')
(1649, '2020-10-04 09:38:42', '82', '3')
(1650, '2020-10-

In [33]:
calorie_insert = '''INSERT INTO calorie (time_id, dateTime, Calories)
                    VALUES (%s,%s,%s);
                '''
# Retrieve my connection and cursor for use in all my inserts
dbconn, cursor = connect_SQLite()
with open('sample_calories.csv') as file:
    time_stamps = csv.DictReader(file)
    value_id = 0
    for time in time_stamps:
        value_id += 1
        record = (value_id, time['dateTime'], time['Calories'])
        print(record)
        cursor.execute(calorie_insert, record)
        dbconn.commit()
        
close_DB_resources(dbconn, cursor)

Connected to heartcaltrip
(1, '2020-10-04 06:00:00', '4.72')
(2, '2020-10-04 06:01:00', '5.66')
(3, '2020-10-04 06:02:00', '5.19')
(4, '2020-10-04 06:03:00', '1.18')
(5, '2020-10-04 06:04:00', '1.18')
(6, '2020-10-04 06:05:00', '1.18')
(7, '2020-10-04 06:06:00', '1.18')
(8, '2020-10-04 06:07:00', '1.18')
(9, '2020-10-04 06:08:00', '1.18')
(10, '2020-10-04 06:09:00', '1.42')
(11, '2020-10-04 06:10:00', '1.18')
(12, '2020-10-04 06:11:00', '1.42')
(13, '2020-10-04 06:12:00', '1.18')
(14, '2020-10-04 06:13:00', '1.18')
(15, '2020-10-04 06:14:00', '1.42')
(16, '2020-10-04 06:15:00', '1.18')
(17, '2020-10-04 06:16:00', '1.18')
(18, '2020-10-04 06:17:00', '1.18')
(19, '2020-10-04 06:18:00', '1.18')
(20, '2020-10-04 06:19:00', '1.18')
(21, '2020-10-04 06:20:00', '1.42')
(22, '2020-10-04 06:21:00', '1.18')
(23, '2020-10-04 06:22:00', '1.18')
(24, '2020-10-04 06:23:00', '1.18')
(25, '2020-10-04 06:24:00', '1.18')
(26, '2020-10-04 06:25:00', '1.18')
(27, '2020-10-04 06:26:00', '1.18')
(28, '2020-

In [34]:
trip_insert = '''INSERT INTO trip (time_id, name, address, description, TimeSpan_begin, TimeSpan_end, longitude, latitude, LineString_coordinates)
                 VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s);
                '''
# Retrieve my connection and cursor for use in all my inserts
dbconn, cursor = connect_SQLite()
with open('denver_trip.csv') as file:
    time_stamps = csv.DictReader(file)
    value_id = 0
    for time in time_stamps:
        value_id += 1
        record = (value_id, time['\ufeffname'], time['address'], time['description'], time['TimeSpan_begin'], time['TimeSpan_end'], time['longitude'], time['latitude'], time['LineString_coordinates'])
        print(record)
        cursor.execute(trip_insert, record)
        dbconn.commit()
        
close_DB_resources(dbconn, cursor)

Connected to heartcaltrip
(1, 'Home (701 Roosevelt Blvd)', '701 Roosevelt Blvd, Freedom, PA 15042', 'Building from 2020-10-03T00:21:12.327Z to 2020-10-03T08:22:40.539Z. Distance 0m', '2020-10-03T00:21:12.327Z', '2020-10-03T08:22:40.539Z', '-80.1614584', '40.6882963', '')
(2, 'Driving', '', 'Driving from 2020-10-03T08:22:40.539Z to 2020-10-03T09:01:36.000Z. Distance 40103m', '2020-10-03T08:22:40.539Z', '2020-10-03T09:01:36.000Z', '', '', '-80.1549814,40.6805847,0 -80.15042449999999,40.675959,0 -80.15022799999998,40.6753216,0 -80.1504375,40.6726753,0 -80.15032459999999,40.672378900000005,0 -80.14822549999998,40.6701755,0 -80.14629469999997,40.6697043,0 -80.14367189999999,40.66832840000001,0 -80.14113989999998,40.6661973,0 -80.13802209999997,40.6650261,0 -80.13633499999997,40.6646723,0 -80.13396769999997,40.6634572,0 -80.13176979999997,40.6619856,0 -80.13026189999998,40.6617219,0 -80.12906849999997,40.661219200000005,0 -80.12832789999997,40.6607311,0 -80.12374959999997,40.6607824,0 -80.12

InvalidTextRepresentation: invalid input syntax for type double precision: ""
LINE 2: ...0-10-03T08:22:40.539Z','2020-10-03T09:01:36.000Z','','','-80...
                                                             ^


In [ ]:
select_sql = '''
                SELECT *
                    FROM heart
                    WHERE address = '1441 Little Raven St, Denver, CO 80202';
            '''

dbconn, cursor = connect_SQLite()
cursor.execute(select_sql)
rec = cursor.fetchall()
for r in rec:
    print(r)

close_DB_resources(dbconn, cursor)